In [1]:
import numpy as np
import os
import time
import requests
import json
import redis
from redis.sentinel import Sentinel

In [6]:
def print_memory_info(redis_client):
    memory_info = redis_client.info('memory')
    print(f"Current Redis memory usage: {memory_info['used_memory_human']}")
    print(f"Peak memory usage: {memory_info['used_memory_peak_human']}")
    print(f"Memory limit: {memory_info['maxmemory_human']}")

def print_eviction_policy(redis_client):
    current_policy = redis_client.config_get('maxmemory-policy')
    print(f"Current policy: {current_policy}")
    
def get_evicted_cnt(redis_client):
    stats = redis_client.info()
    current_evicted = stats['evicted_keys']
    return current_evicted


In [3]:
sentinel = Sentinel([('redis-sentinel', 26379)], socket_timeout=0.1)
master_client = sentinel.master_for('mymaster', socket_timeout=0.1)    

master_info = sentinel.discover_master('mymaster')
print(f"Master: {master_info[0]}:{master_info[1]}")
print_memory_info(master_client)

print_eviction_policy(master_client)

slaves = sentinel.discover_slaves('mymaster')
for slave_host, slave_port in slaves:
    print(f"\nSlave: {slave_host}:{slave_port}")
    slave_client = redis.Redis(host=slave_host, port=slave_port, socket_timeout=0.1)    
    print_memory_info(slave_client)
    print_eviction_policy(slave_client)

Master: 172.28.0.2:6379
Current Redis memory usage: 1.68M
Peak memory usage: 1.68M
Memory limit: 3.00M
Current policy: {'maxmemory-policy': 'allkeys-lru'}

Slave: 172.28.0.4:6379
Current Redis memory usage: 1.59M
Peak memory usage: 1.59M
Memory limit: 0B
Current policy: {'maxmemory-policy': 'noeviction'}


In [8]:
for i in range(20):
    key = 'test_key_' + str(i)
    try:
        write_result = master_client.set(key, 'x' * 1024 * 100)  # 1KB of 'x' characters        
        read_result = master_client.get(key)
        print(f"Write/read {key}, evicted: {get_evicted_cnt(master_client)}")
    except Exception as e:
        print(f"Error writing {key} to Redis: {e}")


Write/read test_key_0, evicted: 60
Write/read test_key_1, evicted: 61
Write/read test_key_2, evicted: 62
Write/read test_key_3, evicted: 63
Write/read test_key_4, evicted: 64
Write/read test_key_5, evicted: 65
Write/read test_key_6, evicted: 66
Write/read test_key_7, evicted: 67
Write/read test_key_8, evicted: 68
Write/read test_key_9, evicted: 69
Write/read test_key_10, evicted: 70
Write/read test_key_11, evicted: 71
Write/read test_key_12, evicted: 72
Write/read test_key_13, evicted: 73
Write/read test_key_14, evicted: 74
Write/read test_key_15, evicted: 75
Write/read test_key_16, evicted: 76
Write/read test_key_17, evicted: 77
Write/read test_key_18, evicted: 78
Write/read test_key_19, evicted: 79
